In [269]:
import pandas as pd
import numpy as np

In [270]:
mass = pd.read_csv('../source/mmALL_073120_csv.csv')
mass.head(2)

,id,country,ccode,year,region,protest,protestnumber,startday,startmonth,startyear,...,protesterdemand4,stateresponse1,stateresponse2,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes
0,201990001,Canada,20,1990,North America,1,1,15.0,1.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...
1,201990002,Canada,20,1990,North America,1,2,25.0,6.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...


In [316]:
print (f'There are { len(set(list(mass["country"]))) } countries in the mass protest dataframe.')

There are 166 countries in the mass protest dataframe.


---
Load Prosperity Index data from first sheet of Prosperity Excel file.

In [271]:
prosperity_df = pd.read_excel('../data/Legatum_Prosperity_Index_Full_2020_Data_Set.xlsx', sheet_name=0)

In [276]:
print (f'The Prosperity Index contains scores on {prosperity_df.shape[0]} countries.')

The Prosperity Index contains scores on 167 countries.


In [277]:
prosperity_df.head(3)

,area_name,area_code,area_group,rank_2007,rank_2008,rank_2009,rank_2010,rank_2011,rank_2012,rank_2013,...,score_2011,score_2012,score_2013,score_2014,score_2015,score_2016,score_2017,score_2018,score_2019,score_2020
0,Afghanistan,AFG,Asia-Pacific,163,163,162,161,162,161,160,...,34.018804,33.529762,34.848416,35.455467,33.878693,34.122529,34.704901,33.234143,33.607822,34.350055
1,Albania,ALB,Eastern Europe,74,75,76,67,66,65,66,...,57.100940,57.345438,57.155699,57.095382,57.409473,57.699371,57.855046,59.182053,60.132551,59.573636
2,Algeria,DZA,Middle East and North Africa,107,109,107,107,103,110,111,...,50.207155,48.905687,48.401319,49.657271,49.944818,50.026566,49.876850,50.919671,51.913637,51.869120


Create a new dataframe containing only the country name, and the overall Prosperity Index score from 2016 (the most recently calculated score.)

In [278]:
country_PI_df = prosperity_df[['area_name','score_2020']].copy()
country_PI_df.columns = ['country', 'prosperity_2020']

In [279]:
country_PI_df.head()

,country,prosperity_2020
0,Afghanistan,34.350055
1,Albania,59.573636
2,Algeria,51.869120
3,Angola,38.816856
4,Argentina,60.487355


The country names in the Prosperity Index does not all match the 166 countries in the larger mass protest dataframe.

Loop through both dataframes to identify any discrepancies.

In [280]:
prosperity_countries = sorted(list(prosperity_df['area_name']))
mass_countries = sorted(list(mass['country'].value_counts().index))

mass_not_prosperity = []
prosperity_not_in_mass = []

for country in mass_countries:
    if country not in prosperity_countries:
        mass_not_prosperity.append(country)
        
for country in prosperity_countries:
    if country not in mass_countries:
        prosperity_not_in_mass.append(country)
        
mass_not_prosperity

['Bhutan',
 'Bosnia',
 'Cape Verde',
 'Congo Brazzaville',
 'Congo Kinshasa',
 'Czech Republic',
 'Czechoslovakia',
 'Gambia',
 'Germany East',
 'Germany West',
 'Ivory Coast',
 'Kosovo',
 'Macedonia',
 'North Korea',
 'Serbia and Montenegro',
 'Slovak Republic',
 'Swaziland',
 'Taiwan',
 'Timor Leste',
 'USSR',
 'United Arab Emirate',
 'Yugoslavia']

In [314]:
#prosperity_not_in_mass

Bhutan barely has any protests.. drop?  Or use same score as Nepal?

In [282]:
mass[mass['country']=='Bhutan'].shape

(31, 31)

Rename Bosnia and Herzegovina to just Bosnia to match mass protest dataframe.

In [284]:
country_PI_df[country_PI_df['country']=='Bosnia and Herzegovina']

,country,prosperity_2020
16,Bosnia and Herzegovina,57.998273


In [285]:
country_PI_df['country'] = country_PI_df['country'].str.replace('Bosnia and Herzegovina', 'Bosnia')

Cape Verde is also known as Cabo Verde. The mass dataframe contains Cape Verde

In [286]:
country_PI_df['country'] = country_PI_df['country'].str.replace('Cabo Verde', 'Cape Verde')

Mass protest dataframe contains both:
* Congo Brazzaville
* Congo Kinshasa

The Prosperity Index contains values for:
* Congo
* Democratic Republic of the Congo

Since Kinshasa is the capital of the Democratic Republic of the Congo, rename the Prosperity Index label as Congo Kinshasa.

Brazzaville is the capital of the Republic of the Congo, so rename the Prosperity Index label to Congo Brazzaville

In [287]:
#country_PI_df['country'] = country_PI_df['country'].str.replace('Congo', 'Congo Brazzaville')
country_PI_df.loc[country_PI_df['country'] == 'Congo' , 'country'] = 'Congo Brazzaville'
                
country_PI_df['country'] = country_PI_df['country'].str.replace('Democratic Republic of Congo', 'Congo Kinshasa')

Mass protest dataframe contains both:

* Czech Republic
* Czechoslovakia
while Prosperity contains a single Czechia score.

Perform same operations as to the Congo score.
Assign the single Czechia score to Czech Republic, then create a new row with the same Prosperity score as Czechoslovakia.

In [288]:
country_PI_df['country'] = country_PI_df['country'].str.replace('Czechia', 'Czech Republic')

#Create new row for Czechoslovakia
country_PI_df.loc[country_PI_df.shape[0]] = ['Czechoslovakia' , country_PI_df[country_PI_df['country']=='Czech Republic']['prosperity_2020'].item() ]

In [289]:
country_PI_df.tail(3)

,country,prosperity_2020
165,Zambia,47.509684
166,Zimbabwe,42.977587
167,Czechoslovakia,73.124612


Gambia is identified as The Gambia in the Prosperity Index. Rename to simply Gandia to merge correctly with the mass protest dataframe.

In [290]:
country_PI_df['country'] = country_PI_df['country'].str.replace('The Gambia', 'Gambia')

Mass protest dataframe contains protest data for:
* Germany East
* Germany West
* Germany

while Prosperity contains a single Germany score.

Create records for East and West Germany scores for a clean merge operation later.

In [291]:
#Create new row for Germany East
country_PI_df.loc[country_PI_df.shape[0]] = ['Germany East' , country_PI_df[country_PI_df['country']=='Germany']['prosperity_2020'].item() ]

#Create new row for Germany West
country_PI_df.loc[country_PI_df.shape[0]] = ['Germany West' , country_PI_df[country_PI_df['country']=='Germany']['prosperity_2020'].item() ]

Ivory Coast is identified as Côte d'Ivoire in the Prosperity Index. Rename to merge correctly with the mass protest dataframe.

In [292]:
country_PI_df['country'] = country_PI_df['country'].str.replace("Côte d'Ivoire", 'Ivory Coast')

Independence of Kosovo is disputed. Assign it Serbia's Prosperity score?

https://en.wikipedia.org/wiki/International_recognition_of_Kosovo

North Macedonia was known as Macedonia until February 2019.  Rename North Macedonia in Prosperity Index to Macedonia to match with mass protest dataframe.
https://en.wikipedia.org/wiki/North_Macedonia

In [293]:
country_PI_df['country'] = country_PI_df['country'].str.replace('North Macedonia', 'Macedonia')

North Korea does not have a Prosperity score.  From the 2017 Foreward:
'..because of conflict or oppression we do not have the data to report on them.
These include Myanmar, North Korea and Syria'

https://prosperitysite.s3-accelerate.amazonaws.com/3515/1187/1128/Legatum_Prosperity_Index_2017.pdf

Create a score with an average of the minimum and the mean to get approximately 25%?  Or one standard deviation below the mean? Assuming normal distribution this would be approx to 34%

In [294]:
country_PI_df.describe()

,prosperity_2020
count,170.000000
mean,57.876721
std,13.627296
min,27.888813
25%,47.012843
50%,56.997148
75%,67.449734
max,84.368117


In [295]:
country_PI_df['prosperity_2020'].mean() - np.std(country_PI_df['prosperity_2020'])

44.28956472876912

Mass protest contains protest data for the following:
* Serbia
* Montenegro
* Serbia and Montenegro

The Prosperity Index only contains two scores, one for Serbia and one for Montenegro.  Both values are similar, but create an average of the two for a composite score.

serbia_score = country_PI_df[country_PI_df['country']=='Serbia']['prosperity_2020'].item()
montenegro_score = country_PI_df[country_PI_df['country']=='Montenegro']['prosperity_2020'].item()
avg_prosperity_serb_mont = (serbia_score + montenegro_score) / 2
avg_prosperity_serb_mont

In [296]:
#Create new row forSerbia and Montenegro
country_PI_df.loc[country_PI_df.shape[0]] = ['Serbia and Montenegro' , avg_prosperity_serb_mont ]

The mass protest dataframe contains values under 'Slovak Republic' vs 'Slovakia' in the Prosperity Index

In [297]:
country_PI_df['country'] = country_PI_df['country'].str.replace('Slovakia', 'Slovak Republic')

The mass protest dataframe values for 'Swaziland' are identified as 'Eswatini' in the Prosperity Index

In [298]:
country_PI_df['country'] = country_PI_df['country'].str.replace('Eswatini', 'Swaziland')

Rename Taiwan, China to simply Taiwan

In [299]:
country_PI_df['country'] = country_PI_df['country'].str.replace('Taiwan, China', 'Taiwan')

There is no Prosperity Index for Timor-Leste, also known as East Timor. Became a sovereign state in 2002 after being under Indonesian rule. Accounts for 0.02% of world's population.  22 rows in mass protest data. Drop?

In [300]:
mass.shape

(17145, 31)

Use same Prosperity score for USSR as Russia.

In [301]:
#Create new row for USSR
country_PI_df.loc[country_PI_df.shape[0]] = ['USSR' , country_PI_df[country_PI_df['country']=='Russia']['prosperity_2020'].item() ]

The mass protest dataframe values for 'United Arab Emirate' are identified as 'United Arab Emirates' in the Prosperity Index

In [302]:
country_PI_df['country'] = country_PI_df['country'].str.replace('United Arab Emirates', 'United Arab Emirate')

Yugoslavia has no current Prosperity score, but could be determined by former countries that made up Yugoslavia.
https://en.wikipedia.org/wiki/Yugoslavia

* Bosnia
* Croatia
* Macedonia
* Serbia
* Slovenia

In [303]:
country_PI_df[country_PI_df['country'].str.startswith('Sl')]

,country,prosperity_2020
131,Slovak Republic,69.625246
132,Slovenia,74.563150


In [304]:
yugo_countries = ['Bosnia', 'Croatia', 'Macedonia', 'Serbia', 'Slovenia']
yugo_mean = np.mean([country_PI_df[country_PI_df['country']==country]['prosperity_2020'].item() for country in yugo_countries])
yugo_mean

64.53108133022

In [305]:
#Create new row for Yugoslavia
country_PI_df.loc[country_PI_df.shape[0]] = ['Yugoslavia' , yugo_mean ]

Double check that there aren't any countries in the mass protest dataframe that do not have a matching Prosperity Index score

In [306]:
prosperity_countries = sorted(list(country_PI_df['country']))
mass_countries = sorted(list(mass['country'].value_counts().index))

mass_not_prosperity = []
prosperity_not_in_mass = []

for country in mass_countries:
    if country not in prosperity_countries:
        mass_not_prosperity.append(country)
        
for country in prosperity_countries:
    if country not in mass_countries:
        prosperity_not_in_mass.append(country)
        
mass_not_prosperity

['Bhutan', 'Kosovo', 'North Korea', 'Timor Leste']

In [307]:
prosperity_not_in_mass

['Australia',
 'Belize',
 'Hong Kong',
 'Iceland',
 'Israel',
 'Malta',
 'New Zealand',
 'Seychelles',
 'São Tomé and Príncipe',
 'Trinidad and Tobago',
 'United States']

Drop all rows from Prosperity Index that are not in the mass protest dataframe.

In [ ]:
#sorted(list(country_PI_df['country']))

In [308]:
for country_to_drop in prosperity_not_in_mass:
    country_PI_df.drop( country_PI_df[country_PI_df['country'] == country_to_drop].index, inplace=True)


In [311]:
len(list(country_PI_df['country']))

162

In [313]:
len(set(list(mass['country'])))

166

Merge into larger mass protest dataframe.

In [317]:
mass = mass.merge(country_PI_df, how='left', 
                        left_on='country', 
                        right_on='country')
mass.head(2)

,id,country,ccode,year,region,protest,protestnumber,startday,startmonth,startyear,...,stateresponse1,stateresponse2,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes,prosperity_2020
0,201990001,Canada,20,1990,North America,1,1,15.0,1.0,1990.0,...,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...,79.819703
1,201990002,Canada,20,1990,North America,1,2,25.0,6.0,1990.0,...,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...,79.819703
